# Working with LLMs SDKs

In [ ]:
! pip install openai

In [32]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv


# Load environment variables from .env file
#load_dotenv()

from pathlib import Path

env_path = r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\Packt Bootcamp\code\.env"
load_dotenv(dotenv_path=env_path, override=True)




True

In [33]:
# Access the environment variables
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
tavily_api_key = os.getenv("TAVILY_API_KEY")
hf_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")



In [22]:
#if using Azure OpenAI
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

In [ ]:
# if using OpenAI
from openai import OpenAI
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [23]:
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What's the highest mountain?"}
    ]
)

print(response.choices[0].message.content)

The highest mountain in the world is Mount Everest, which is located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China. Its peak rises to an elevation of 8,848.86 meters (29,031.7 feet) above sea level, making it the tallest mountain on Earth.


## Hugging face hub

In [ ]:
%pip install --upgrade --quiet huggingface_hub langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [18]:
from huggingface_hub import InferenceClient

client = InferenceClient(model="deepseek-ai/DeepSeek-R1-0528", provider="auto", token=hf_api_token)
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "Hello"}]
)
print(completion.choices[0].message)


ChatCompletionOutputMessage(role='assistant', content='<think>\nHmm, the user just said "Hello". That\'s a simple greeting, but it\'s a good start! \n\nOkay, first things first - they\'re probably testing if I\'m responsive or just initiating a casual interaction. The tone seems neutral, maybe even slightly friendly. \n\nI should keep my response warm but open-ended since they haven\'t stated any specific needs yet. A smiley would make it feel more welcoming. \n\nI wonder if they\'re new here? The single-word message feels like someone testing the waters. Better avoid overwhelming them with options though - just a simple "How can I help?" invitation leaves the ball in their court. \n\n...Wait, should I have mirrored their exact greeting? No, "Hi there" feels more natural than repeating "Hello". The exclamation point adds energy without being overbearing. \n\nNoting that they used proper capitalization - probably not in a hurry then. Keeping my response equally polished.\n</think>\nHi t

## Inspecting response's structure

In [24]:
import json

# If it's a Pydantic-based object (which it is in OpenAI SDK)
response_dict = response.model_dump()  # or response.dict() if using an older version of Pydantic

# Pretty-print the JSON
print(json.dumps(response_dict, indent=2))


{
  "id": "chatcmpl-BkZKax9IXYAmxXceBBwSLclaKjdOs",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The highest mountain in the world is Mount Everest, which is located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China. Its peak rises to an elevation of 8,848.86 meters (29,031.7 feet) above sea level, making it the tallest mountain on Earth.",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
         

In [25]:
import json
from datetime import datetime


# Extract data
choice = response_dict["choices"][0]
message = choice["message"]["content"]
finish_reason = choice["finish_reason"]
model = response_dict["model"]
timestamp = datetime.utcfromtimestamp(response_dict["created"]).strftime('%Y-%m-%d %H:%M:%S UTC')
tokens = response_dict["usage"]

# Pretty print with emojis
print("🧠 **Model**: ", model)
print("🕒 **Timestamp**: ", timestamp)
print("✅ **Finish Reason**: ", finish_reason)
print("\n💬 **Assistant's Message**:\n")
print("🗻", message)

print("\n📊 **Token Usage**:")
print(f"   🔹 Prompt tokens: {tokens['prompt_tokens']}")
print(f"   🔹 Completion tokens: {tokens['completion_tokens']}")
print(f"   🔹 Total tokens: {tokens['total_tokens']}")

# Optional: Add content filter check
filters = choice["content_filter_results"]
print("\n🔍 **Safety Checks**:")
for category, result in filters.items():
    status = "🟢 Safe" if not result["filtered"] else "🔴 Filtered"
    print(f"   - {category.capitalize()}: {status}")


🧠 **Model**:  gpt-4o-2024-08-06
🕒 **Timestamp**:  2025-06-20 16:56:40 UTC
✅ **Finish Reason**:  stop

💬 **Assistant's Message**:

🗻 The highest mountain in the world is Mount Everest, which is located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China. Its peak rises to an elevation of 8,848.86 meters (29,031.7 feet) above sea level, making it the tallest mountain on Earth.

📊 **Token Usage**:
   🔹 Prompt tokens: 22
   🔹 Completion tokens: 65
   🔹 Total tokens: 87

🔍 **Safety Checks**:
   - Hate: 🟢 Safe
   - Self_harm: 🟢 Safe
   - Sexual: 🟢 Safe
   - Violence: 🟢 Safe


C:\Users\vaalt\AppData\Local\Temp\ipykernel_41204\2058013866.py:10: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  timestamp = datetime.utcfromtimestamp(response_dict["created"]).strftime('%Y-%m-%d %H:%M:%S UTC')


In [7]:
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "List me all the ingredients to produce drug."}
    ]
)

print(response.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}

## Interactive

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

# Start the chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        print("Ending the chat. Goodbye!")
        break

    # Add user message
    messages.append({"role": "user", "content": user_input})

    # Call the model
    response = client.chat.completions.create(
        model="gpt-4o",  # or your deployment name
        messages=messages
    )

    assistant_reply = response.choices[0].message.content
    print(f"Assistant: {assistant_reply}")

    # Add assistant message to history
    messages.append({"role": "assistant", "content": assistant_reply})

Assistant: Hello! How can I assist you today?
Assistant: The highest mountain in the world is Mount Everest. Its peak reaches an elevation of 8,848.86 meters (29,031.7 feet) above sea level. Mount Everest is part of the Himalayas and is located on the border between Nepal and the Tibet Autonomous Region of China.
Assistant: Ah, I see! If we're considering the tallest mountain in the entire solar system, that title goes to Olympus Mons on Mars. Olympus Mons is an enormous shield volcano and stands about 13.6 miles (22 kilometers) high, which is nearly three times the height of Mount Everest. Its diameter is approximately 370 miles (600 kilometers), making it one of the largest volcanoes in terms of area as well.
Ending the chat. Goodbye!


# Building your firt ReAct Agent with LangChain

In [26]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import requests
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent



# Initialize the Azure OpenAI model
llm = AzureChatOpenAI(
    openai_api_version=openai_api_version,
    azure_deployment=azure_chat_deployment,
)

# Create a human message and invoke the model
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
response = llm.invoke([message])

# Print the response
print(response)

content="J'adore programmer." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BkZoHoQ59imWPLJFD5henr5nF5LpO', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'

## Search tool

In [27]:
tavily_tool = TavilySearchResults(max_results=5, tavily_api_key=tavily_api_key)

## Portfolio tool

In [28]:
import json

@tool
def read_sample_portfolio(json_path: str = "sample_portfolio.json") -> str:
    """
    Reads the sample_portfolio.json file and returns its content as a string.
    Each entry includes the stock symbol, sector, quantity, purchase price, and purchase date.
    """
    if not os.path.exists(json_path):
        return f"File not found: {json_path}"

    with open(json_path, "r") as f:
        portfolio = json.load(f)

    if not isinstance(portfolio, list):
        return "Unexpected portfolio format."

    response = "Sample Portfolio:\n"
    for stock in portfolio:
        response += (
            f"- {stock['symbol']} ({stock['sector']}): "
            f"{stock['quantity']} shares @ ${stock['purchase_price']} "
            f"(Bought on {stock['purchase_date']})\n"
        )
    return response

## Build the agent

In [29]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

prompt = f"""
You are a financial advisor. You will be provided with a sample portfolio of stocks.
Your task is to analyze the portfolio and provide insights on its performance, diversification, and any potential improvements.
Always use the current date {datetime.today().strftime('%Y-%m-%d')} for any calculations or assessments.
"""

In [30]:
memory = MemorySaver()
tools = [tavily_tool, read_sample_portfolio]
agent_executor = create_react_agent(llm, tools, checkpointer=memory, prompt=prompt)

In [31]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# Config for the run
config = {"configurable": {"thread_id": "abc123"}}

# Helper: Pretty-print messages with formatting and emojis
def pretty_print_step(step):
    print("\n" + "="*30)
    for message in step.get("messages", []):
        if isinstance(message, HumanMessage):
            print("🧑‍💬 \033[1mUser:\033[0m", message.content)
        elif isinstance(message, AIMessage):
            print("🤖 \033[1mAI:\033[0m", message.content)
        elif isinstance(message, ToolMessage):
            print(f"🛠️ \033[1mTool [{message.name}]:\033[0m {message.content}")
        else:
            print("📦 \033[1mOther:\033[0m", message)

    # Optional: handle tool calls (if not part of messages)
    if "tool_calls" in step:
        print("\n📡 \033[1mTool Calls:\033[0m")
        for call in step["tool_calls"]:
            print(f"🔧 Tool: {call['name']}")
            print(f"🆔 Call ID: {call['id']}")
            print(f"📥 Args: {call['args']}\n")

# Stream agent responses with pretty output
for step in agent_executor.stream(
    {
        "messages": [
            HumanMessage(content="given the current financial market, how can I optimize my portfolio?")
        ],
        "config": config,
    },
    config,
    stream_mode="values",
):
    pretty_print_step(step)



🧑‍💬 User: given the current financial market, how can I optimize my portfolio?

🧑‍💬 User: given the current financial market, how can I optimize my portfolio?
🤖 AI: 

🧑‍💬 User: given the current financial market, how can I optimize my portfolio?
🤖 AI: 
🛠️ Tool [read_sample_portfolio]: Sample Portfolio:
- AAPL (Technology): 13 shares @ $1202.57 (Bought on 2022-03-12)
- GOOGL (Technology): 21 shares @ $1625.97 (Bought on 2022-01-02)
- MSFT (Technology): 68 shares @ $2579.45 (Bought on 2022-10-05)
- AMZN (Consumer Discretionary): 86 shares @ $1604.04 (Bought on 2022-10-26)
- TSLA (Consumer Discretionary): 15 shares @ $944.23 (Bought on 2022-08-19)
- JNJ (Healthcare): 67 shares @ $2739.44 (Bought on 2022-06-23)
- NVDA (Technology): 72 shares @ $475.52 (Bought on 2022-12-17)
- XOM (Energy): 93 shares @ $2592.5 (Bought on 2022-04-23)
- META (Communication Services): 100 shares @ $215.68 (Bought on 2022-02-27)
- V (Financials): 93 shares @ $1105.12 (Bought on 2022-09-21)
- PG (Consumer Stapl